In [183]:
from sklearn.metrics import  accuracy_score
from sklearn.datasets import load_iris
from itertools import combinations
import numpy as np
import pandas as pd

In [184]:
class logisticregression():
    def __init__(self,train_data,train_labels,lr=0.01,batch_size=None,epoch=10,print_every = 10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.print_every = print_every
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        return np.dot(x.T,(y_pred-y))
    
    def train(self):
        for i in range(self.epoch):
            y_pred = self.sigmoid(np.dot(self.train_data,self.params))
            loss = self.cost(self.train_labels,y_pred)
            
            gra = self.gradient(self.train_labels,y_pred,self.train_data)
            
            self.params -= self.lr*gra
            
            if self.print_every:
                if i%self.print_every == 0 or i == self.epoch-1:
                    print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = self.sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result >= 0.5 ] = 1
        result[result < 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [191]:
def cross_validate(data,k=5):
    if len(data)%k != 0:
        print('cant vsplit',len(data),' by ',k)
        return
    
    data_splitted = np.vsplit(data,k)
    aggrigate_result = []
    for i in range(len(data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(data_splitted)) if j !=i ]
        test = data_splitted[i]
        for item in items:
            if len(train) == 0:
                train = data_splitted[item]
            else:
                train = np.concatenate((train,data_splitted[item]), axis=0)
            
        logistic = logisticregression(train[:,:-1],train[:,-1:],epoch=10,print_every=None)
        logistic.train()
        
        result = logistic.evaluate(test[:,:-1],test[:,-1:])
        aggrigate_result.append(result)
        
    return aggrigate_result

In [203]:
data = load_iris()

data = pd.concat((pd.DataFrame(data['data']),pd.DataFrame(data['target'])),axis=1)
data.columns= ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)','target']
data = data[(data['target']==0) | (data['target']==1)]
data_np = data.values
np.random.shuffle(data_np)

In [205]:
cross_validate(data_np,2)

[0.98, 0.96]